In [ ]:
import ollama

prompt = (
    "Hello World."
)

response = ollama.chat(model='gpt-oss:20b', messages=[
    {'role': 'user', 'content': prompt}
])

print(response['message']['content'])

Hello! 👋  
It looks like you’ve sent the classic “Hello World” message. How can I help you today? Whether you’re looking for information, a quick answer, or a deeper dive into a topic, just let me know!


In [6]:
import pandas as pd

df_claims = pd.read_csv('causal_claims_processed.csv')

df_claims

,Paper_ID,Title,Year,Cause_JEL,Cause_Description,Effect_JEL,Effect_Description,Cause_Text,Effect_Text,Claim
0,w0117,Earnings Growth on the Job and Between Jobs,1980,J62,Labor and Demographic Economics: General; Mobi...,O49,"Economic Development, Innovation, Technologica...",job mobility,earnings growth,job mobility -> earnings growth
1,w0117,Earnings Growth on the Job and Between Jobs,1980,J62,Labor and Demographic Economics: General; Mobi...,O49,"Economic Development, Innovation, Technologica...",job mobility,earnings growth,job mobility -> earnings growth
2,w0117,Earnings Growth on the Job and Between Jobs,1980,J62,Labor and Demographic Economics: General; Mobi...,O49,"Economic Development, Innovation, Technologica...",job mobility,earnings growth,job mobility -> earnings growth
3,w0117,Earnings Growth on the Job and Between Jobs,1980,J62,Labor and Demographic Economics: General; Mobi...,O49,"Economic Development, Innovation, Technologica...",job mobility,earnings growth,job mobility -> earnings growth
4,w0117,Earnings Growth on the Job and Between Jobs,1980,J62,Labor and Demographic Economics: General; Mobi...,O49,"Economic Development, Innovation, Technologica...",job mobility,earnings growth,job mobility contributes significantly to earn...
...,...,...,...,...,...,...,...,...,...,...
995,w17326,Cycles of Wage Discrimination,2011,J62,Labor and Demographic Economics: General; Mobi...,J29,Labor and Demographic Economics: General; Dema...,job movers,dynamic response to labor market conditions,job movers -> dynamic response to labor market...
996,w17559,A Sorted Tale of Globalization: White Collar J...,2011,F69,International Economics: General; Economic Imp...,J62,Labor and Demographic Economics: General; Mobi...,Rising service imports from China and India,Downward occupational switching,Rising service imports from China and India ->...
997,w17559,A Sorted Tale of Globalization: White Collar J...,2011,F69,International Economics: General; Economic Imp...,J62,Labor and Demographic Economics: General; Mobi...,Rising service imports from China and India,Downward occupational switching,Rising service imports from China and India ->...
998,w17559,A Sorted Tale of Globalization: White Collar J...,2011,F69,International Economics: General; Economic Imp...,J62,Labor and Demographic Economics: General; Mobi...,Rising service imports from China and India,Downward occupational switching,Rising service imports from China and India ->...


In [17]:
# From the causal_claims_processed.csv file, import the cause and effect pairs and their JEL codes.
#cause_effect_jel = df_claims[['Cause_Description', 'Effect_Description', 'Cause_JEL', 'Effect_JEL']].dropna()

# Create a prompt to send to the LLM. Use the cause and effect pairs and their JEL codes.
cause_effect_jel = df_claims[['Cause_Description', 'Effect_Description', 'Cause_JEL', 'Effect_JEL']].dropna()

batch_size = 10
prompts = []
for i in range(0, len(cause_effect_jel), batch_size):
    batch = cause_effect_jel.iloc[i:i+batch_size]
    pairs_text = ""
    for _, row in batch.iterrows():
        pairs_text += (
            f"Cause (JEL: {row['Cause_JEL']}): {row['Cause_Description']}\n"
            f"Effect (JEL: {row['Effect_JEL']}): {row['Effect_Description']}\n\n"
        )
    prompt = (
        "Try to link causal pairs. The data will be batched in several queries\n"
        f"Here is the data for batch number {i/10}:\n"
        "Cause and Effect pairs based on JEL codes:\n"
        f"{pairs_text}"
    )
    prompts.append(prompt)

# Example: print the first batch prompt
print(prompts[1])

Try to link causal pairs. The data will be batched in several queries
Here is the data for batch number 1.0:
Cause and Effect pairs based on JEL codes:
Cause (JEL: J62): Labor and Demographic Economics: General; Mobility, Unemployment, Vacancies, and Immigrant Workers: General; Job, Occupational, and Intergenerational Mobility; Promotion; Guideline: Covers studies about issues related to non-geographic labor mobility including promotion.; Keywords: Career Change, Demotion, Intergenerational Mobility, Job Transfer, Labor Mobility, Non-geographic Mobility, Occupational Mobility, Promotion, Social Mobility
Effect (JEL: I26): Health, Education, and Welfare: General; Education and Research Institutions: General; Returns to Education; Guideline: Covers studies about issues related to returns to education at both micro and macro levels. Studies about human capital returns to education should be cross-classified under J24.; Keywords: Educational Outcomes, Human Capital, Human Capital Investmen

In [18]:
from tqdm import tqdm

responses = []
for prompt in tqdm(prompts, desc="Processing prompts"):
    response = ollama.chat(model='gpt-oss:20b', messages=[
        {'role': 'user', 'content': prompt}
    ])
    responses.append(response['message']['content'])


Processing prompts:  43%|████▎     | 43/100 [1:12:00<1:35:27, 100.48s/it]


ResponseError: error parsing tool call: raw='def flip_substring(s, l, r):
    return s[:l] + ''.join('1' if c=='0' else '0' for c in s[l:r+1]) + s[r+1:]

s='1010010111000010'
n=len(s)
best=None
best_interval=None
for l in range(n):
    for r in range(l, n):
        t=flip_substring(s, l, r)
        if best is None or t<best:
            best=t
            best_interval=(l,r)
best, best_interval
', err=invalid character 'd' looking for beginning of value (status code: 500)

In [ ]:
#loop through the prompts and print them

for response in responses:
    print(response)

    #figure out why it's thinking so long. maybe make it trace the graph. what makes it so time consuming?
    #Test with a complicated question.

**Causal Mapping for Batch 0.0**

| Cause (JEL) | Effect (JEL) | Notes |
|-------------|--------------|-------|
| **J62 – Labor & Demographic Economics (Mobility, Promotion, etc.)** | **O49 – Economic Development, Innovation & Growth** | The same cause is paired with O49 **eight times** in the batch, indicating a strong (repeated) linkage. |
| **J62 – Labor & Demographic Economics (Mobility, Promotion, etc.)** | **I26 – Health, Education & Welfare (Returns to Education)** | The same cause is paired with I26 **twice** in the batch, showing a secondary linkage to human‑capital outcomes. |

**Summary**

- The J62 code, which focuses on non‑geographic labor mobility and promotion, is consistently linked to two broad outcome domains in the data set:
  1. **Economic Growth & Aggregate Productivity (O49)** – the dominant causal path.
  2. **Human‑Capital Returns to Education (I26)** – a secondary, but still present, causal path.

These links can be used to identify the primary and secondary e